# Connect to local mongo

In [2]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")

db = client["abortion"]

# Preprocess

In [3]:
from nltk.tokenize import TweetTokenizer
import re
import unidecode

# TweetTokenizer basically unterstands arrows, smiley faces and weird punctuation
tokenizer = TweetTokenizer(preserve_case=True, reduce_len=True, strip_handles=False)


def my_preprocess(text, keep_hashtags=True):
    toks = tokenizer.tokenize(text)

    ret = []
    for tok in toks:
        if tok[0] == "#" and not keep_hashtags:
            continue
        if tok[:4] == "http":
            continue
        if tok[0] == "@":
            continue
        # removing numbers and punctuation
        if tok.isnumeric():
            continue
        ret.append(unidecode.unidecode(tok.lower()))
    return " ".join(ret)

In [11]:
for t in db.abortion.find({},{"tweet.id", "tweet.text", "tweet.full_text", "tweet.retweeted_status"}):
    text = ""
    tweet = t["tweet"]
    if("full_text" in tweet):
        text = tweet["full_text"]
    elif("retweeted_status" in tweet and "full_text" in tweet["retweeted_status"]):
        text = tweet["retweeted_status"]["full_text"]
    elif("retweeted_status" in tweet and "text" in tweet["retweeted_status"]):
        text = tweet["retweeted_status"]["text"]
    elif("text" in tweet):
        text = tweet["text"]
    else:
        print(tweet)
    db.abortion.find_one_and_update({"tweet.id": tweet["id"]}, {"$set": {"tweet.full_text": my_preprocess(text)}})

It takes a while to run

After it finished, run db.abortion.count({"tweet.full_text": {$exists: true}}) on mongo shell and it should say 613550